In [ ]:
# Fixing a temporary visualisation problem on Google Colab
!pip uninstall -y  pandas
!pip install pandas

Installing the required packages, i.e., PyTerrier and its DeepImpact plugin.

In [ ]:
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_deepimpact.git

Starting PyTerrier.

In [1]:
import pyterrier as pt
pt.init()

PyTerrier 0.7.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


Importing the DeepImpact custom PyTerrier's indexer.

In [2]:
from pyt_deepimpact import DeepImpactIndexer

Indexing the Vaswani collection with PyTerrier, exploiting a pre-trained DeepImpact model, automatically downloaded from the Web.

In [3]:
vaswani = pt.get_dataset("vaswani")

pt_index_path = './terrier_di_vaswani'

parent = pt.index.IterDictIndexer(pt_index_path)
parent.setProperty("termpipelines", "")
indexer = DeepImpactIndexer(parent, batch_size=32)
indexer.index(vaswani.get_corpus_iter())

index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
index_di = pt.IndexFactory.of(index_ref)


Downloading...
From: https://drive.google.com/uc?id=17I2TWCB2hBSQ-E0Yt2sBEDH2z_rV0BN0
To: /home/ubuntu/.pyterrier/pyt_deepimpact/d37a5e7c46233dbc95c9438960317418
 15%|█▌        | 201M/1.32G [00:05<00:22, 49.2MB/s][Errno 28] No space left on device


Loading checkpoint /home/ubuntu/.pyterrier/pyt_deepimpact/d37a5e7c46233dbc95c9438960317418


 15%|█▌        | 203M/1.32G [00:07<00:39, 28.2MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing MultiBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing MultiBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MultiBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MultiBERT were not initialized from the model checkpoint at bert-base-uncased and are n

[Oct 18, 11:19:31] #> Loading checkpoint /home/ubuntu/.pyterrier/pyt_deepimpact/d37a5e7c46233dbc95c9438960317418


FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/.pyterrier/pyt_deepimpact/d37a5e7c46233dbc95c9438960317418'

Indexing the Vaswani collection with PyTerrier, for classical BM25 retrieval.

In [ ]:
pt_index_path = './terrier_vaswani'

indexer = pt.index.IterDictIndexer(pt_index_path)
indexer.setProperty("termpipelines", "")
index_ref = indexer.index(vaswani.get_corpus_iter())

index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
index = pt.IndexFactory.of(index_ref)

Performing the retrieval experiment

In [ ]:
pt.Experiment([
        pt.BatchRetrieve(index, wmodel="BM25"),
        pt.BatchRetrieve(index_di, wmodel="Tf")
    ],
    vaswani.get_topics(), vaswani.get_qrels(),
    names=['bm25', "deep_impact"],
    eval_metrics=["map", "recip_rank", "ndcg_cut_10"]
)